In [1]:
from pathlib import Path
import pandas as pd
import sys

# Exploratory Data Analysis (EDA)



## Data Loading


In [2]:
from pathlib import Path
import pandas as pd
import sys

# update python src path
ROOT = Path().resolve().parent
if str(ROOT / "src") not in sys.path:
    sys.path.insert(0, str(ROOT / "src"))

print(ROOT)
from data_workflow.viz import bar_sorted, time_line, histogram_chart, save_fig
from data_workflow.utils import bootstrap_diff_means
from data_workflow.io import read_parquet

# set needed paths
processed_path = ROOT / "data/processed"
figs_path = ROOT / "reports/figures"
figs_path.mkdir(parents=True, exist_ok=True)

print(processed_path.relative_to(ROOT))# use relative path to hide my device path :)
print(figs_path.relative_to(ROOT))


/Users/mohammedbalkhair/bootcamp/Week-2_Project
data/processed
reports/figures


In [3]:
# load data
df = pd.read_parquet(processed_path/"analytics_table.parquet")
df.head()


,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount__is_outlier
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,2025-11-15,False
1,A0002,0002,8.135,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,2025-11-20,False
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,2025-11-22,<NA>
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,2025-11-15,False
4,A0005,0004,97.75,1,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,2025-11-25,True


## 1- Audit data


In [4]:
# Basic info
print("rows", len(df))

print("columns", len(df.columns))


rows 5
columns 17


In [5]:
print("\ntypes:")
print(df.dtypes)


types:
order_id                   string[python]
user_id                    string[python]
amount                            Float64
quantity                            Int64
created_at            datetime64[ns, UTC]
status                             object
status_clean                       object
amount__isna                         bool
quantity__isna                       bool
date                               object
year                              float64
month                      string[python]
dow                                object
hour                              float64
country                            object
signup_date                        object
amount__is_outlier                boolean
dtype: object


In [6]:
print("\n Missings:")
print(df.isna().sum())


 Missings:
order_id              0
user_id               0
amount                1
quantity              1
created_at            1
status                0
status_clean          0
amount__isna          0
quantity__isna        0
date                  1
year                  1
month                 1
dow                   1
hour                  1
country               0
signup_date           0
amount__is_outlier    1
dtype: int64


##  questions


### question 1: what is the total amount for each country?



In [7]:

amount_by_country = df.groupby("country")["amount"].sum()

print(amount_by_country)
fig = bar_sorted(amount_by_country, "country", "amount", "Total amount by Country")
save_fig(fig, figs_path / "amount_by_country.png")
fig


country
AE        0.0
SA    143.385
Name: amount, dtype: Float64


A: SA only have an amount , the reason is that we do not have enough data for AE


### Question 2: How does revenue trend over time (monthly)?

Identifying monthly trends helps understand seasonality and business growth patterns.


In [8]:
amount_by_weekday = df.groupby("dow", dropna=False)["amount"].sum().reset_index()
print(amount_by_weekday)

fig = time_line(amount_by_weekday, "dow", "amount", "week day trend")
save_fig(fig, figs_path / "amount_trend_weekday.png")
fig


         dow  amount
0     Monday  20.635
1    Tuesday     0.0
2  Wednesday    25.0
3        NaN   97.75


A: we can see that the amount of orders in tuesday is 0 , we can not build an assumption becasue there are orders without date maybe some of them are in tuesday

### Question 3: what is the distribution of order amounts?



In [9]:
fig = histogram_chart(df, "amount", nbins=30, title="Distribution of Order Amounts")
save_fig(fig, figs_path / "amount_distribution.png")
fig


In [10]:
print(df["amount"].describe())


count          4.0
mean      35.84625
std      41.883474
min          8.135
25%       11.40875
50%          18.75
75%        43.1875
max          97.75
Name: amount, dtype: Float64


A : we have a clear outlier which is 97 and it is affecting the mean adn which will affect on std 

## Summary

Key findings:
1. the amount for AE country was 0 as all amount we have is for SA
2. we are not welling to spend on tuesday
3. we have an outlier which affect in the amount of stats

**Caveats:**
- we dont have enough data for AE country 
- some dates were missing and it has a significant amount
- we cannot estimate the amount distribution using only 5 rows, the stats will be soo sensitive and not reliable
